In [1]:
import numpy as np, PIL.Image as Image, matplotlib.pyplot as plt, os, pandas as pd, datasets, torch
from utility_scripts.pose_estimation_utilities import euler_angles_to_matrix, matrix_to_euler_angles, draw_axis

In [2]:
datasets_dict = {
    'face_recognition' : [datasets.MS1MV2()],
    'emotion_recognition' : [datasets.AffectNet(), datasets.RAFDB()],
    'age_gender_race' : [datasets.MORPH(), datasets.UTKFace(), datasets.FairFace()],
    'attribute_recognition' : [datasets.CelebA()],
    'head_pose' : [datasets.W300LP()] 
}


In [4]:
loader = datasets.get_balanced_loader(datasets_dict, batch_size = 1024, num_workers = 2, epoch_size = 10000)

In [5]:
images, labels = next(iter(loader))

In [6]:
images.shape

torch.Size([1024, 3, 112, 112])

In [7]:
(labels['face_recognition'] != -1).sum()

tensor(213)

In [8]:
(labels['emotion'] != -1).sum(), np.unique(labels['emotion'], return_counts = True)[1][1:]

(tensor(169), array([23, 26, 26, 23, 18, 27, 26]))

In [9]:
(labels['age'] != -1).sum()

tensor(132)

In [10]:
(labels['gender'] != - 1).sum(), np.unique(labels['gender'], return_counts = True)[1][1:]

(tensor(203), array([112,  91]))

In [11]:
(labels['race'] != -1).sum(), np.unique(labels['race'], return_counts = True)[1][1:]

(tensor(141), array([32, 32, 18, 30, 29]))

In [12]:
attributes = labels['attributes']
template = torch.ones(40, dtype = torch.int32) * -1
mask = torch.logical_not(torch.all(attributes == template, dim = 1))
sum(mask)

tensor(211)

In [13]:
poses = labels['pose']
template = torch.ones(3, dtype = torch.float32) * -999
mask = torch.logical_not(torch.all(poses == template, dim = 1))
sum(mask)

tensor(228)